## Teemad ja nende märgendamine lausetele

Süntaksianalüüsi tulemusel saadi sagedusloend lauses nimeüksuste alluvateks (children) olevatest omadussõnalistest täienditest (amod, mis kirjeldavad nimeüksuse omadusi), nimeüksuste ülemuseks (parent) olevatest verbidest (kirjeldavad, mida nimeüksus lauses teeb) ning nende ülemuste alluvateks olevatest määrustest (advmod, mis kirjeldavad tegevuse viisi, kohta, aega vms). Järgnevalt tuleb jaotada laused klassidesse.

In [1]:
from estnltk import Text
from estnltk.taggers.system.rule_taggers import AmbiguousRuleset, Ruleset, StaticExtractionRule
from estnltk.taggers import PhraseTagger
import sqlite3
from estnltk.converters import json_to_text
from estnltk.converters import text_to_json
import csv
from collections import Counter

### I temaatilised märksõnad ja laused

Temaatilised märksõnad, mida märgendamisel kasutatakse on salvestatud CSV-faili.

In [2]:
with open('thematic_keywords.csv', encoding='UTF-8') as csv_file:
    rows = []
    reader = csv.DictReader(csv_file)
    for row in reader:
        rows.append(row)

In [3]:
len(rows)

97

In [4]:
print(rows)

[{'lemma': 'callable', 'class': 'string'}, {'lemma': "'vastanduma',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'võitlema',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'häbistama',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'vaidlema',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'nokkima',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'protestima',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'sõdima',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'debateerima',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'diskuteerima',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'nurisema',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'ründama',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'süüdistama',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'kandideerima',", 'class': 'valimised'}, {'lemma': "'valima',", 'class': 'valimised'}, {'lemma': "'võitma',", 'class': 'valimised'}, {'lemma': "'lubama',", 'class': 'va

Andmebaasist loetakse süntaktilise analüüsi kihiga laused.

In [5]:
sentences = []
sentence_ids = []

In [6]:
con = sqlite3.connect("media_data_complete.db")
cur = con.cursor()

In [7]:
for row in cur.execute("SELECT ID, sentence FROM sentences_syntax_analysis"):
    sentence_id = row[0]
    sentence = json_to_text(json_text=row[1])
    sentences.append(sentence)
    sentence_ids.append(sentence_id)

In [8]:
con.close()

### II PhraseTagger märksõnade lausetele märkimiseks

In [9]:
vocabulary_file = 'thematic_keywords.csv'
ruleset = Ruleset()
ruleset.load(file_name=vocabulary_file, key_column='lemma')

In [10]:
phrasetagger = PhraseTagger(output_layer='keywords',
                      input_layer='morph_analysis',
                      input_attribute='lemma',
                      ruleset=ruleset,
                      output_attributes=['class'],
                      #decorator=decorator,
                      conflict_resolver='KEEP_MAXIMAL',
                      ignore_case=True)

In [11]:
# Lausetele märgitakse peale märksõnade kiht
for sentence in sentences:
    phrasetagger.tag(sentence)

In [13]:
sentences_with_keywords = []

for sentence in sentences:
    if len(sentence.keywords) > 0:
        sentences_with_keywords.append(sentence)

display(sentences_with_keywords[20])
display(sentences_with_keywords[20].keywords)
display(sentences_with_keywords[20].stanza_syntax)

Text(text='See et kui nad väidavad et Andres Siitan ta on rikkunud seadust eelarve distsipliini seadusega siis miks politsei seda asja ei menetle et et kui kui kui asi peenes väidavad et et kolm koma viis miljonit krooni.')

Layer(name='keywords', attributes=('class',), spans=SL[EnvelopingSpan(['menetle'], [{'class': 'valitsemine'}])])

Layer(name='stanza_syntax', attributes=('id', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc', 'parent_span', 'children'), spans=SL[Span('See', [{'id': 1, 'lemma': 'see', 'upostag': 'P', 'xpostag': 'P', 'feats': {'sg': 'sg', 'n': 'n'}, 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]),
Span('et', [{'id': 2, 'lemma': 'et', 'upostag': 'J', 'xpostag': 'J', 'feats': {}, 'head': 5, 'deprel': 'mark', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('kui', [{'id': 3, 'lemma': 'kui', 'upostag': 'J', 'xpostag': 'J', 'feats': {}, 'head': 5, 'deprel': 'mark', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('nad', [{'id': 4, 'lemma': 'tema', 'upostag': 'P', 'xpostag': 'P', 'feats': {'pl': 'pl', 'n': 'n'}, 'head': 5, 'deprel': 'nsubj', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('väidavad', [{'id': 5, 'lemma': 'väitma', 'upostag': 'V', 'xpostag': 'V', 'feats': {'vad': 'vad'}, 'head': 1, 'deprel': 'advcl', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('et', [{'id': 6, 'lemma': 'et', 'upostag': 'J', 'xpostag': 'J', 'feats': {}, 'head': 11, 'deprel': 'mark', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('Andres', [{'id': 7, 'lemma': 'Andres', 'upostag': 'H', 'xpostag': 'H', 'feats': {'sg': 'sg', 'n': 'n'}, 'head': 11, 'deprel': 'nsubj', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('Siitan', [{'id': 8, 'lemma': 'Siitan', 'upostag': 'H', 'xpostag': 'H', 'feats': {'sg': 'sg', 'n': 'n'}, 'head': 7, 'deprel': 'flat', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('ta', [{'id': 9, 'lemma': 'tema', 'upostag': 'P', 'xpostag': 'P', 'feats': {'sg': 'sg', 'n': 'n'}, 'head': 11, 'deprel': 'nsubj', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('on', [{'id': 10, 'lemma': 'olema', 'upostag': 'V', 'xpostag': 'V', 'feats': {'b': 'b'}, 'head': 11, 'deprel': 'aux', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('rikkunud', [{'id': 11, 'lemma': 'rikkunud', 'upostag': 'A', 'xpostag': 'A', 'feats': {'pl': 'pl', 'n': 'n'}, 'head': 5, 'deprel': 'ccomp', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('seadust', [{'id': 12, 'lemma': 'seadus', 'upostag': 'S', 'xpostag': 'S', 'feats': {'sg': 'sg', 'p': 'p'}, 'head': 11, 'deprel': 'obj', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('eelarve', [{'id': 13, 'lemma': 'eelarve', 'upostag': 'S', 'xpostag': 'S', 'feats': {'sg': 'sg', 'g': 'g'}, 'head': 14, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('distsipliini', [{'id': 14, 'lemma': 'distsipliin', 'upostag': 'S', 'xpostag': 'S', 'feats': {'sg': 'sg', 'g': 'g'}, 'head': 15, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('seadusega', [{'id': 15, 'lemma': 'seadus', 'upostag': 'S', 'xpostag': 'S', 'feats': {'sg': 'sg', 'kom': 'kom'}, 'head': 11, 'deprel': 'obl', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('siis', [{'id': 16, 'lemma': 'siis', 'upostag': 'D', 'xpostag': 'D', 'feats': {}, 'head': 22, 'deprel': 'advmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('miks', [{'id': 17, 'lemma': 'miks', 'upostag': 'D', 'xpostag': 'D', 'f

Kuigi märksõna lausest leiti, ei pruugi see olla lauses seotud huvipakkuva isiku/organisatsiooniga. Seetõttu tuleb võtta arvesse ka süntaksianalüüsi kihti, et teha kindlaks, kas leitud märksõna esineb lauses seoses isiku/organisatsiooniga (kas täiendi, verbi või määruse positsioonil).

In [14]:
phrase = []

In [15]:
for entity in sentences_with_keywords[20].entity_phrases:
    for span in entity:
        phrase.append(span)

In [16]:
print(phrase)

[Span('Andres', [{'normalized_text': 'Andres', 'lemma': 'Andres', 'root': 'Andres', 'root_tokens': ['Andres'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'H'}])]


In [17]:
keywrds = []

In [18]:
for word in sentences_with_keywords[20].keywords:
    keywrds.append(word[0])

In [19]:
print(keywrds)

[Span('menetle', [{'normalized_text': 'menetle', 'lemma': 'menetlema', 'root': 'menetle', 'root_tokens': ['menetle'], 'ending': '0', 'clitic': '', 'form': 'o', 'partofspeech': 'V'}])]


In [21]:
for subspan in phrase:
    parent = sentences_with_keywords[20].stanza_syntax.get(subspan)['parent_span']
    parent_word = sentences_with_keywords[20]['words'].get(parent) 
    if parent_word in phrase:
        continue
    parent_morph = sentences_with_keywords[20].morph_analysis.get(parent)
    if 'V' not in parent_morph['partofspeech']:
        continue
    print(parent['lemma'])
    print('Kas on ülemus: ', parent_morph in keywrds)
    
    # Verbist ülemuse alluv, kui on määrus
    parent_children = sentences_with_keywords[20].stanza_syntax.get(parent)['children']
    if len(parent_children) > 1:
        for child in parent_children:
            child_word = sentence['words'].get(child) 
            if child_word in phrase:
                continue
            elif 'advmod' != child['deprel']:
                continue
            if child == keywrds[0]:
                print(child['lemma'])
                print('Kas on ülemuse alluv: ', child == keywrds[0])
                break
            print(child['lemma'])
            print('Kas on ülemuse alluv: ', child == keywrds[0])
            
    children = sentences_with_keywords[20].stanza_syntax.get(subspan)['children']
    if len(children) > 0:
        for child in children:
            child_word = sentences_with_keywords[20]['words'].get(child) 
            if child_word in phrase:
                continue
            elif 'amod' != child['deprel']:
                continue
            if child == keywrds[0]:
                print(child['lemma'])
                print('Kas on alluv: ', child == keywrds[0])
                break
            print(child['lemma'])
            print('Kas on alluv: ', child == keywrds[0])
    
    print(list(s.text for s in phrase))
    print(parent['lemma'])

rikkunud
Kas on ülemus:  False
['Andres']
rikkunud


Kõigil lausetel:

In [22]:
sentences_with_classes = []

In [23]:
for sentence in sentences_with_keywords:
    keywrds = []
    for word in sentence.keywords:
        keywrds.append(word[0])
        
    classes = []
    
    for entity in sentence.entity_phrases:
        phrase = []
        for span in entity:
            phrase.append(span)
    
        for subspan in phrase:
            parent = sentence.stanza_syntax.get(subspan)['parent_span']
            if parent != None:
                # Verbist ülemus
                parent_morph = sentence.morph_analysis.get(parent)
                if 'V' not in parent_morph['partofspeech']:
                    continue
                for i in range(len(keywrds)):
                    if keywrds[i] == parent_morph:
                        cl = sentence['keywords'][i]['class']
                        classes.append(cl)
                # Verbist ülemuse alluv, kui on määrus
                parent_children = sentence.stanza_syntax.get(parent)['children']
                if len(parent_children) > 1:
                    for child in parent_children:
                        if 'advmod' != child['deprel']:
                            continue
                        child_morph = sentence.morph_analysis.get(child)
                        for i in range(len(keywrds)):
                            if keywrds[i] == child_morph:
                                cl = sentence['keywords'][i]['class']
                                classes.append(cl)
                                
            
            # Täiendid
            children = sentence.stanza_syntax.get(subspan)['children']
            if len(children) > 0:
                for child in children:
                    if 'amod' != child['deprel']:
                        continue
                    child_morph = sentence.morph_analysis.get(child)
                    for i in range(len(keywrds)):
                        if keywrds[i] == child_morph:
                            cl = sentence['keywords'][i]['class']
                            classes.append(cl)
    key = tuple(classes)
    value = [sentence]
    if len(classes) > 0:
        sentences_with_classes.append(dict(zip(key, value)))

In [24]:
# Leitud lauseid:
len(sentences_with_classes)

752

In [25]:
display(sentences_with_classes[:5])

[{'valimised': Text(text='Teatas et ta kavatseb ka riigikokku kandideerida kevadel et kuidas ta hakkab seal töötama sotside fraktsioonis kus Eiki nestoriga kestev kas tema elust tõmbuksid linnast oma soojast peadest välja kangust.')},
 {'valimised': Text(text='Näiteks lubaduse mida mina ei mäletanudki IRL lubas enne eelmisi valimisi kahe tuhande kroonist huvialaringid toetust last.')},
 {'vaidlused ja erimeelsused': Text(text='Keskerakond reformierakond omaette vastanduma ja on hinnatud isegi üllatas et et et ma ise olin maha maganud et Margus Tsahkna seda teemat niimoodi tõstatas ja tegelikult ka IRLile on see.')},
 {'valitsemine': Text(text='Selle asemel et ma kunagi kunagi küsisin ka kohtasin tänaval laval Andres tarandit ja siis ka Indrek tarand oli Euroopa parlamendi liikmest pärast pikka mari mineerimist otsustanud ennast.')},
 {'valitsemine': Text(text='Miks riigikogu riigikogu koori juhtima.')}]

In [38]:
vaidl_erim_kokku = 0
valimised_kokku = 0
valetamine_kokku = 0
valitsemine_kokku = 0

In [39]:
for key in sentences_with_classes:
    if 'vaidlused ja erimeelsused' in key:
        vaidl_erim_kokku += 1
    elif 'valimised' in key:
        valimised_kokku += 1
    elif 'valetamine' in key:
        valetamine_kokku += 1
    elif 'valitsemine' in key:
        valitsemine_kokku += 1

In [41]:
print("vaidlused ja erimeelsused:", vaidl_erim_kokku)
print("valimised:", valimised_kokku)
print("valetamine:", valetamine_kokku)
print("valitsemine:", valitsemine_kokku)

vaidlused ja erimeelsused: 108
valimised: 333
valetamine: 14
valitsemine: 297
